In [5]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd "/content/gdrive/MyDrive/Capstone/t2s/
from utilites import *
!pip install deeppavlov
!pip install transformers
from deeppavlov import build_model
template_matching_model = build_model('query_pr', download=True)

!pip install finbert-embedding
from finbert_embedding.embedding import FinbertEmbedding
finbert = FinbertEmbedding()

!pip install spacy
!pip install xgboost
!pip install sister
!pip install fuzzywuzzy
!pip install spacy[transformers]
!python -m spacy download en_core_web_trf


from google.colab import drive
drive.mount("/content/gdrive")

import spacy
import numpy as np

nlp = spacy.load("en_core_web_sm")

import re
import csv
import pandas as pd
import bs4
import requests
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')

from spacy.matcher import Matcher 
from spacy.tokens import Span 


from tqdm import tqdm


import xgboost as xgb
from xgboost.sklearn import XGBClassifier

import pandas as pd
import spacy
import string
import re
import sister
import pickle
import numpy as np


from scipy.spatial.distance import cosine
from fuzzywuzzy import fuzz


file_name = "/content/gdrive/MyDrive/Capstone/t2s/XGB_template_classifier.pkl"

# load
XGB_template_classifier = pickle.load(open(file_name, "rb"))



with open("/content/gdrive/MyDrive/Capstone/t2s/relations_embeddings.pkl", "rb") as f: # "rb" because we want to read in binary mode
    relation_embeddings = pickle.load(f)

with open("/content/gdrive/MyDrive/Capstone/t2s/entities_embeddings.pkl", "rb") as f: # "rb" because we want to read in binary mode
    entity_embeddings = pickle.load(f)

import json

# loading ontologies
with open("/content/gdrive/MyDrive/Capstone/t2s/allennlp_ontology.json", 'r') as f:
  ontology = json.load(f)
relations=list(ontology["relation"].keys())

entities_ontology=list(ontology["entities"].keys())

bert_template={4:["SELECT (count(*) as? count) WHERE {<http://crypto.org/ENTITY_CLASS/ENTITY><http://crypto.org/RELATION_CLASS/RELATION> ?o.}",
                  "SELECT (count(*) as? count) WHERE {?s <http://crypto.org/RELATION_CLASS/RELATION> <http://crypto.org/ENTITY_CLASS/ENTITY>.}"]
            ,5:""}
XGB_template={2:[1,"SELECT DISTINCT ?o WHERE { <http://crypto.org/ENTITY_CLASS/ENTITY> <http://crypto.org/RELATION_CLASS/RELATION> ?o.}",
                 "SELECT DISTINCT ?s WHERE { ?s <http://crypto.org/RELATION_CLASS/RELATION> <http://crypto.org/ENTITY_CLASS/ENTITY>.}"],
           16:[2,"SELECT DISTINCT ?o WHERE { <http://crypto.org/SUBJECT1_CLASS/SUBJECT1><http://crypto.org/RELATION1_CLASS/RELATION1> ?o. <http://crypto.org/SUBJECT2_CLASS/SUBJECT2><http://crypto.org/RELATION2_CLASS/RELATION2> ?o. }"],
           305:[2,""]}
ner = spacy.load('en_core_web_trf')


In [ ]:
def text2sparql(query):
  sparql_query_list=[]

  template_id,n_entities, query_templates=template_classification(query)
  
  if template_id==1:
    relation=get_single_relation(query)
    
    if relation=="Not found":
      relation_class="Not found"
    else:
      relation_class=ontology["relation"][relation]
    entity=get_single_entity(query)
    if entity == "Not found":
      entity_class=="Not found"
    else:
      entity_class=ontology["entities"][entity]
    
    if (entity!="Not found" and relation!="Not found" ):
      for query_template in query_templates:
        sparql_query_list.append(query_template.replace("ENTITY_CLASS",entity_class.replace(" ","_")).replace("ENTITY",entity.replace(" ","_")).replace("RELATION_CLASS",relation_class.replace(" ","_")).replace("RELATION",relation.replace(" ","_")))
    else:
      sparql_query_list.append("Not found")

  if n_entities==100:
    entities=get_entities(query)
    if len(entities)!=0:
      for entity in entities:
        entity_class=ontology["entities"][entity]
        sparql_query_list.append(query_template.replace("ENTITY_CLASS",entity_class.replace(" ","_")).replace("ENTITY",entity.replace(" ","_")))
    else:
      sparql_query_list.append("Not found")

  
  return sparql_query_list


query1="What are the papers that Ramit Sawhney authored?"
query2="How many papers has Ramit Sawhney authored?"

text2sparql(query1)